In [1]:
import os
from backoff import on_exception, expo
import json
import requests
from collections import Counter
from datetime import datetime
from ratelimit import limits, sleep_and_retry, RateLimitException
import time
import urllib.request
import asyncio
import aiohttp
import nest_asyncio
from async_timeout import timeout
from requests.exceptions import HTTPError

In [2]:
data_path = "D:\\METIS\\data\\"
image_path = "D:\\METIS\\images\\"

images_saved = []
met_objects = []

In [3]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects'
response = requests.get(url, allow_redirects=True)
all_objects = json.loads(response.text)
objectIDs = all_objects['objectIDs']

In [4]:
@sleep_and_retry
@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_object_details(object_id, img_url=True):
    url ='https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(object_id)
    response = requests.get(url, timeout=10)
    response_json = json.loads(response.text)
    met_objects.append(response_json)
    
    if img_url:
        return response_json['primaryImage']

@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_image(object_id, url):
    filename = image_path + "{}.jpg".format(object_id)
    im = urllib.request.urlretrieve(url, filename)
    images_saved.append({
                        'objectID': object_id, 
                        'url': url,
                        'filename': filename
                        })

In [5]:
img_url = get_object_details(1567)
get_image(1567, img_url)

In [6]:
images_saved

[{'objectID': 1567,
  'url': 'https://images.metmuseum.org/CRDImages/ad/original/79684.jpg',
  'filename': 'D:\\METIS\\images\\1567.jpg'}]

In [7]:
images_saved = []
met_objects = []

start = datetime.now()

for i, object_id in enumerate(objectIDs):
    if i % 1000 == 0:
        with open(data_path + "images_saved.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass
   # print(i, object_id)

In [8]:
start = datetime.now()

for i, object_id in enumerate(objectIDs[320000:340000]):
    if i % 1000 == 0:
        with open(data_path + "images_saved_320.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects_320.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i + 320000, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass

320000: 0:00:00.006981 elapsed
321000: 0:17:17.740166 elapsed
322000: 0:32:23.259781 elapsed
323000: 0:51:00.967241 elapsed
324000: 1:06:36.015522 elapsed
325000: 1:23:07.204405 elapsed
326000: 1:41:08.796518 elapsed
327000: 1:59:23.131125 elapsed
328000: 2:14:53.593452 elapsed
329000: 2:31:44.361359 elapsed
330000: 2:50:25.607675 elapsed
331000: 3:06:40.873105 elapsed
332000: 3:23:21.752080 elapsed
333000: 3:43:45.192118 elapsed
334000: 3:59:27.151439 elapsed
335000: 4:15:31.994551 elapsed
336000: 4:35:19.465507 elapsed
337000: 4:52:07.438159 elapsed
338000: 5:09:33.214608 elapsed
339000: 5:29:46.619679 elapsed


In [9]:
with open(data_path + "images_saved_320.json", "w") as write_file:
    json.dump(images_saved, write_file)   
            
with open(data_path + "met_objects_320.json", "w") as write_file:
    json.dump(met_objects, write_file)  

In [10]:
len(images_saved)

3909